In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# -------------------------------------------------
# 1. Load data
# -------------------------------------------------
df = pd.read_csv("final_processed_for_xgboost.csv")

# We model log_transfer_fee instead of raw transfer_fee
target_col = "log_transfer_fee"

# -------------------------------------------------
# 2. Drop ID / text columns that we do NOT want as features
#    (pure identifiers, names, or all-NaN)
# -------------------------------------------------
drop_feature_cols = [
    "transfer_fee",           # raw target
    "log_transfer_fee",       # target itself
    "player_id",
    "from_club_id",
    "to_club_id",
    "from_club_name",
    "to_club_name",
    "player_name",
    "transfer_date",
    "transfer_season",
    "date_of_birth",
    "contract_expiration_date",
    "from_domestic_competition_id",
    "to_domestic_competition_id",
    "from_total_market_value",  # all NaN in your file
    "to_total_market_value",    # all NaN in your file
    "market_value_in_eur"       # we keep only log_market_value_in_eur
]

feature_df = df.drop(columns=[c for c in drop_feature_cols if c in df.columns])

# -------------------------------------------------
# 3. Select numeric / boolean features
# -------------------------------------------------
numeric_cols = feature_df.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()
# Make sure target is not inside
numeric_cols = [c for c in numeric_cols if c != target_col]

# Remove one dummy per group to avoid perfect collinearity
baseline_dummies = [
    "position_Midfield",          # baseline for position
    "sub_position_Centre-Back",   # baseline for sub-position
    "foot_right",                 # baseline for foot
    "sub_position_Goalkeeper"     # redundant with position_Goalkeeper
]
numeric_cols = [c for c in numeric_cols if c not in baseline_dummies]

X = df[numeric_cols]
y = df[target_col]

# -------------------------------------------------
# 4. Train–test split
# -------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -------------------------------------------------
# 5. Build pipeline: impute → scale → linear regression
# -------------------------------------------------
pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

pipe.fit(X_train, y_train)




,steps,"[('imputer', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [3]:
# -------------------------------------------------
# 6. Evaluate on test set
# -------------------------------------------------
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# -------------------------------------------------
# 6. Evaluate on test set
# -------------------------------------------------
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test, y_pred)  # no 'squared' arg in older sklearn
rmse = np.sqrt(mse)                       # convert MSE -> RMSE manually
r2 = r2_score(y_test, y_pred)

print(f"Test MSE (log fee): {mse:.3f}")
print(f"Test RMSE (log fee): {rmse:.3f}")
print(f"Test R^2: {r2:.3f}")


# -------------------------------------------------
# 7. Inspect coefficients
# -------------------------------------------------
lin = pipe.named_steps["model"]
coefs = lin.coef_

coef_df = pd.DataFrame({
    "feature": numeric_cols,
    "coef_std": coefs
})
coef_df["abs_coef"] = coef_df["coef_std"].abs()
coef_df = coef_df.sort_values("abs_coef", ascending=False)

print(coef_df.head(20))

Test MSE (log fee): 1.066
Test RMSE (log fee): 1.033
Test R^2: 0.639
                            feature  coef_std  abs_coef
33          log_market_value_in_eur  1.240472  1.240472
9                       minutes_365  0.266608  0.266608
14                        games_365 -0.239480  0.239480
8          to_foreigners_percentage  0.135964  0.135964
34                    transfer_year  0.119308  0.119308
2               contract_years_left  0.104450  0.104450
18                 minutes_per_game -0.080065  0.080065
19                position_Defender  0.064161  0.064161
5        from_foreigners_percentage  0.062365  0.062365
22      sub_position_Centre-Forward  0.054155  0.054155
11                      assists_365  0.052861  0.052861
0                      height_in_cm  0.045757  0.045757
25         sub_position_Left Winger  0.040409  0.040409
21    sub_position_Central Midfield  0.038747  0.038747
31                        foot_both -0.037800  0.037800
10                        goals_365

In [6]:
import pandas as pd
import statsmodels.api as sm

# 1. Load data
df = pd.read_csv("final_processed_for_xgboost.csv")

# 2. Choose target
target = "log_transfer_fee"
y = df[target]

# 3. Drop ID / text columns you don't want as predictors
drop_cols = [
    "transfer_fee",
    "log_transfer_fee",
    "player_id",
    "from_club_id",
    "to_club_id",
    "from_club_name",
    "to_club_name",
    "player_name",
    "transfer_date",
    "transfer_season",
    "date_of_birth",
    "contract_expiration_date",
    "from_domestic_competition_id",
    "to_domestic_competition_id",
    "from_total_market_value",   # all NaN
    "to_total_market_value"      # all NaN
]

drop_cols += ["market_value_in_eur"]


feature_df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# 4. Avoid dummy trap: drop one category for each one-hot group
#    (these will become the "baseline" categories)
dummy_baselines = [
    "position_Midfield",             # baseline position
    "sub_position_Central Midfield", # baseline sub-position
    "foot_right"                     # baseline foot
]
for col in dummy_baselines:
    if col in feature_df.columns:
        feature_df = feature_df.drop(columns=col)

# 5. Keep only numeric columns as predictors
X = feature_df.select_dtypes(include=["int64", "float64"])

# 6. Add intercept term
X = sm.add_constant(X)

# 7. Fit multiple linear regression with robust (HC3) SE
model = sm.OLS(y, X, missing="drop")
results = model.fit(cov_type="HC3")

print(results.summary())


                            OLS Regression Results                            
Dep. Variable:       log_transfer_fee   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.763
Method:                 Least Squares   F-statistic:                     551.3
Date:                Sat, 15 Nov 2025   Prob (F-statistic):               0.00
Time:                        23:00:47   Log-Likelihood:                -4143.7
No. Observations:                3806   AIC:                             8331.
Df Residuals:                    3784   BIC:                             8469.
Df Model:                          21                                         
Covariance Type:                  HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [7]:
"""
Optimized multiple linear regression for transfer fee prediction.

- Target: log_transfer_fee
- Data: final_processed_for_xgboost.csv
"""

import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score


# ============================================================
# 1. Load data
# ============================================================
df = pd.read_csv("final_processed_for_xgboost.csv")

# Ensure target exists and drop rows with missing target
if "log_transfer_fee" not in df.columns:
    raise ValueError("Column 'log_transfer_fee' not found in the CSV.")

df = df.dropna(subset=["log_transfer_fee"])
y = df["log_transfer_fee"].copy()


# ============================================================
# 2. Choose a cleaned feature set to reduce multicollinearity
#    (optimized MLR feature design)
# ============================================================
# We intentionally:
# - keep log_market_value_in_eur, drop raw market_value_in_eur
# - keep per-90 rates (goals_per90, assists_per90, cards_per90)
#   instead of totals where possible
# - keep total minutes_365 as a proxy for trust/usage
# - keep key club/context variables and contract features

base_features = [
    "height_in_cm",
    "age_at_transfer",
    "contract_years_left",
    "from_foreigners_percentage",
    "to_foreigners_percentage",
    "to_squad_size",
    "to_average_age",
    "minutes_365",
    "goals_per90",
    "assists_per90",
    "cards_per90",
    "log_market_value_in_eur",
    "transfer_year",
]

# Check that all required columns exist
missing = [c for c in base_features if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}")

# Restrict to numeric features (just in case)
df_feat = df[base_features].select_dtypes(include=["int64", "float64"]).copy()


# ============================================================
# 3. Optional: center some features for numerical stability
#    and a more interpretable intercept
# ============================================================
center_cols = ["height_in_cm", "age_at_transfer", "transfer_year"]

for col in center_cols:
    mean_val = df_feat[col].mean()
    df_feat[col + "_c"] = df_feat[col] - mean_val

# Build final OLS feature list (using centered versions)
ols_features = [
    "height_in_cm_c",
    "age_at_transfer_c",
    "contract_years_left",
    "from_foreigners_percentage",
    "to_foreigners_percentage",
    "to_squad_size",
    "to_average_age",
    "minutes_365",
    "goals_per90",
    "assists_per90",
    "cards_per90",
    "log_market_value_in_eur",
    "transfer_year_c",
]

X_ols = df_feat[ols_features].copy()

# Drop rows with any missing predictor values
valid_idx = X_ols.dropna().index
X_ols = X_ols.loc[valid_idx]
y_ols = y.loc[valid_idx]

# Add intercept term
X_ols = sm.add_constant(X_ols)


# ============================================================
# 4. Fit optimized multiple linear regression (OLS)
# ============================================================
ols_model = sm.OLS(y_ols, X_ols)
ols_results = ols_model.fit(cov_type="HC3")  # robust SE for heteroskedasticity

print("\n=========== OLS (Optimized MLR) Summary ===========\n")
print(ols_results.summary())


# ============================================================
# 5. Interpret key coefficients programmatically (optional)
#    We convert some coefficients to percentage effects
# ============================================================
def pct_effect(beta, delta=1.0):
    """
    Approximate percentage effect on fee for a change 'delta' in the predictor.
    """
    return (np.exp(beta * delta) - 1.0) * 100


print("\n=========== Key Coefficient Interpretations ===========\n")

for name in ["age_at_transfer_c", "height_in_cm_c", "contract_years_left",
             "log_market_value_in_eur", "transfer_year_c"]:
    if name in ols_results.params.index:
        b = ols_results.params[name]
        if name == "age_at_transfer_c":
            print(f"Age (+1 year): {pct_effect(b, 1):.2f}% change in fee")
        elif name == "height_in_cm_c":
            print(f"Height (+5 cm): {pct_effect(b, 5):.2f}% change in fee")
        elif name == "contract_years_left":
            print(f"Contract (+1 year): {pct_effect(b, 1):.2f}% change in fee")
        elif name == "log_market_value_in_eur":
            # For log(MV), a doubling is +ln(2) in log(MV)
            delta = np.log(2.0)
            print(f"Market value (x2): {pct_effect(b, delta):.2f}% change in fee")
        elif name == "transfer_year_c":
            print(f"Year (+1 year): {pct_effect(b, 1):.2f}% change in fee")


# ============================================================
# 6. Optional: Regularized MLR (Ridge) for better prediction
# ============================================================
# Here we use the same cleaned features (without the constant)
print("\n=========== Ridge Regression (for prediction) ===========\n")

X = df_feat[ols_features].dropna()
y_ridge = y.loc[X.index]

X_train, X_test, y_train, y_test = train_test_split(
    X, y_ridge, test_size=0.2, random_state=42
)

# RidgeCV will choose the best alpha via cross-validation
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

ridge = RidgeCV(alphas=alphas, cv=5)
ridge_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", ridge),
])

ridge_pipe.fit(X_train, y_train)

y_pred = ridge_pipe.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Best alpha (Ridge): {ridge_pipe.named_steps['ridge'].alpha_}")
print(f"Test MSE (log fee): {mse:.4f}")
print(f"Test RMSE (log fee): {rmse:.4f}")
print(f"Test R^2 (log fee): {r2:.4f}")

# Coefficients on standardized scale
ridge_coefs = ridge_pipe.named_steps["ridge"].coef_

coef_table = pd.DataFrame({
    "feature": ols_features,
    "coef_std": ridge_coefs
})
coef_table["abs_coef"] = coef_table["coef_std"].abs()
coef_table = coef_table.sort_values("abs_coef", ascending=False)

print("\nTop Ridge coefficients (standardized features):\n")
print(coef_table[["feature", "coef_std"]].head(15))



=========== OLS (Optimized MLR) Summary ===========

                            OLS Regression Results                            
Dep. Variable:       log_transfer_fee   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     860.8
Date:                Sat, 15 Nov 2025   Prob (F-statistic):               0.00
Time:                        23:02:35   Log-Likelihood:                -4146.1
No. Observations:                3806   AIC:                             8320.
Df Residuals:                    3792   BIC:                             8408.
Df Model:                          13                                         
Covariance Type:                  HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------

In [27]:
"""
Optimized multiple linear regression for transfer fee prediction
+ local explanations per player (row-level contributions).

- Target: log_transfer_fee
- Data: final_processed_for_xgboost.csv
"""

import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score


# ============================================================
# 1. Load data
# ============================================================
df = pd.read_csv("final_processed_for_xgboost.csv")

# Ensure target exists and drop rows with missing target
if "log_transfer_fee" not in df.columns:
    raise ValueError("Column 'log_transfer_fee' not found in the CSV.")

df = df.dropna(subset=["log_transfer_fee"])
y = df["log_transfer_fee"].copy()


# ============================================================
# 2. Choose a cleaned feature set to reduce multicollinearity
#    (optimized MLR feature design)
# ============================================================
base_features = [
    "height_in_cm",
    "age_at_transfer",
    "contract_years_left",
    "from_foreigners_percentage",
    "to_foreigners_percentage",
    "to_squad_size",
    "to_average_age",
    "minutes_365",
    "goals_per90",
    "assists_per90",
    "cards_per90",
    "log_market_value_in_eur",
    "transfer_year",
]

# Check that all required columns exist
missing = [c for c in base_features if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}")

# Restrict to numeric features (just in case)
df_feat = df[base_features].select_dtypes(include=["int64", "float64"]).copy()


# ============================================================
# 3. Center some features for numerical stability
# ============================================================
center_cols = ["height_in_cm", "age_at_transfer", "transfer_year"]

for col in center_cols:
    mean_val = df_feat[col].mean()
    df_feat[col + "_c"] = df_feat[col] - mean_val

# Build final OLS feature list (using centered versions)
ols_features = [
    "height_in_cm_c",
    "age_at_transfer_c",
    "contract_years_left",
    "from_foreigners_percentage",
    "to_foreigners_percentage",
    "to_squad_size",
    "to_average_age",
    "minutes_365",
    "goals_per90",
    "assists_per90",
    "cards_per90",
    "log_market_value_in_eur",
    "transfer_year_c",
]

X_ols = df_feat[ols_features].copy()

# Drop rows with any missing predictor values
valid_idx = X_ols.dropna().index
X_ols = X_ols.loc[valid_idx]
y_ols = y.loc[valid_idx]

# Add intercept term
X_ols = sm.add_constant(X_ols)


# ============================================================
# 4. Fit optimized multiple linear regression (OLS)
# ============================================================
ols_model = sm.OLS(y_ols, X_ols)
ols_results = ols_model.fit(cov_type="HC3")  # robust SE for heteroskedasticity

print("\n=========== OLS (Optimized MLR) Summary ===========\n")
print(ols_results.summary())


# ============================================================
# 5. Interpret key coefficients programmatically (optional)
# ============================================================
def pct_effect(beta, delta=1.0):
    """
    Approximate percentage effect on fee for a change 'delta' in the predictor.
    """
    return (np.exp(beta * delta) - 1.0) * 100


print("\n=========== Key Coefficient Interpretations ===========\n")

for name in ["age_at_transfer_c", "height_in_cm_c", "contract_years_left",
             "log_market_value_in_eur", "transfer_year_c"]:
    if name in ols_results.params.index:
        b = ols_results.params[name]
        if name == "age_at_transfer_c":
            print(f"Age (+1 year): {pct_effect(b, 1):.2f}% change in fee")
        elif name == "height_in_cm_c":
            print(f"Height (+5 cm): {pct_effect(b, 5):.2f}% change in fee")
        elif name == "contract_years_left":
            print(f"Contract (+1 year): {pct_effect(b, 1):.2f}% change in fee")
        elif name == "log_market_value_in_eur":
            delta = np.log(2.0)
            print(f"Market value (x2): {pct_effect(b, delta):.2f}% change in fee")
        elif name == "transfer_year_c":
            print(f"Year (+1 year): {pct_effect(b, 1):.2f}% change in fee")


# ============================================================
# 6. Regularized MLR (Ridge) for better prediction (optional)
# ============================================================
print("\n=========== Ridge Regression (for prediction) ===========\n")

X = df_feat[ols_features].dropna()
y_ridge = y.loc[X.index]

X_train, X_test, y_train, y_test = train_test_split(
    X, y_ridge, test_size=0.2, random_state=42
)

alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

ridge = RidgeCV(alphas=alphas, cv=5)
ridge_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", ridge),
])

ridge_pipe.fit(X_train, y_train)

y_pred = ridge_pipe.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Best alpha (Ridge): {ridge_pipe.named_steps['ridge'].alpha_}")
print(f"Test MSE (log fee): {mse:.4f}")
print(f"Test RMSE (log fee): {rmse:.4f}")
print(f"Test R^2 (log fee): {r2:.4f}")

ridge_coefs = ridge_pipe.named_steps["ridge"].coef_

coef_table = pd.DataFrame({
    "feature": ols_features,
    "coef_std": ridge_coefs
})
coef_table["abs_coef"] = coef_table["coef_std"].abs()
coef_table = coef_table.sort_values("abs_coef", ascending=False)

print("\nTop Ridge coefficients (standardized features):\n")
print(coef_table[["feature", "coef_std"]].head(15))


# ============================================================
# 7. Local explanations per transfer (row-level contributions)
# ============================================================
print("\n=========== Building local explanations (per transfer) ===========\n")

# Get parameters (including intercept)
params = ols_results.params  # index: ['const', 'height_in_cm_c', ..., 'transfer_year_c']

# Separate intercept and feature coefficients
intercept = params["const"]
feature_coefs = params.drop(labels=["const"])

# X_ols currently includes the 'const' column; we only want feature columns
X_features = X_ols.drop(columns=["const"])

# Ensure the same order
X_features = X_features[feature_coefs.index]

# Compute contributions: contribution_j = beta_j * x_ij
contrib_df = X_features.multiply(feature_coefs, axis=1)

# Rename contribution columns to make it explicit: coef_<feature_name>
contrib_df = contrib_df.rename(columns={c: f"coef_{c}" for c in contrib_df.columns})

# Add intercept as its own column
contrib_df["coef_intercept"] = intercept

# Predicted log fee from OLS
pred_log_fee = ols_results.predict(X_ols)
contrib_df["pred_log_transfer_fee"] = pred_log_fee

# Predicted fee on original scale
contrib_df["pred_transfer_fee"] = np.exp(pred_log_fee)

# If actual fee exists, add it and residual
if "transfer_fee" in df.columns:
    actual_fee = df.loc[valid_idx, "transfer_fee"]
    contrib_df["actual_transfer_fee"] = actual_fee.values
    contrib_df["residual_log"] = np.log(actual_fee.values + 1e-9) - pred_log_fee

# Attach identifiers (per player / transfer) from original df
id_cols = [c for c in [
    "player_id",
    "player_name",
    "from_club_name",
    "to_club_name",
    "transfer_season",
    "transfer_date"
] if c in df.columns]

id_df = df.loc[valid_idx, id_cols].reset_index(drop=True)

explain_df = pd.concat([id_df, contrib_df.reset_index(drop=True)], axis=1)

# Save per-transfer explanations
per_transfer_path = "mlr_local_explanations_per_transfer.csv"
explain_df.to_csv(per_transfer_path, index=False)
print(f"Per-transfer local explanations saved to: {per_transfer_path}")


# ============================================================
# 8. Collapse to per-player (keep most recent transfer)
# ============================================================
print("\n=========== Collapsing to per-player (latest transfer) ===========\n")

player_df = explain_df.copy()

# Prefer transfer_date if available
if "transfer_date" in player_df.columns:
    player_df["transfer_date"] = pd.to_datetime(player_df["transfer_date"])
    # fill NaT with a very old date so idxmax still works
    player_df["transfer_date_filled"] = player_df["transfer_date"].fillna(
        pd.Timestamp("1900-01-01")
    )
    idx_latest = player_df.groupby("player_id")["transfer_date_filled"].idxmax()
    player_latest = player_df.loc[idx_latest].drop(columns=["transfer_date_filled"])
elif "transfer_season" in player_df.columns:
    # if seasons are strings like '17/18', max() is usually fine
    idx_latest = player_df.groupby("player_id")["transfer_season"].idxmax()
    player_latest = player_df.loc[idx_latest]
else:
    # fallback: if transfer_year exists somewhere in df, attach and use it
    if "transfer_year" in df.columns:
        year_series = df.loc[valid_idx, "transfer_year"].reset_index(drop=True)
        player_df["transfer_year"] = year_series
        idx_latest = player_df.groupby("player_id")["transfer_year"].idxmax()
        player_latest = player_df.loc[idx_latest]
    else:
        # if absolutely nothing, just take the last row per player_id
        idx_latest = player_df.groupby("player_id").tail(1).index
        player_latest = player_df.loc[idx_latest]

player_latest = player_latest.sort_values("player_id")

per_player_path = "mlr_local_explanations_per_player_55.csv"
player_latest.to_csv(per_player_path, index=False)
print(f"Per-player (latest transfer) explanations saved to: {per_player_path}")



=========== OLS (Optimized MLR) Summary ===========

                            OLS Regression Results                            
Dep. Variable:       log_transfer_fee   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     860.8
Date:                Sun, 16 Nov 2025   Prob (F-statistic):               0.00
Time:                        00:12:57   Log-Likelihood:                -4146.1
No. Observations:                3806   AIC:                             8320.
Df Residuals:                    3792   BIC:                             8408.
Df Model:                          13                                         
Covariance Type:                  HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------

In [28]:
explain_df  

,player_id,player_name,from_club_name,to_club_name,transfer_season,transfer_date,coef_height_in_cm_c,coef_age_at_transfer_c,coef_contract_years_left,coef_from_foreigners_percentage,...,coef_goals_per90,coef_assists_per90,coef_cards_per90,coef_log_market_value_in_eur,coef_transfer_year_c,coef_intercept,pred_log_transfer_fee,pred_transfer_fee,actual_transfer_fee,residual_log
0,149729,João Mário,Benfica,Besiktas,25/26,2025-07-01,-0.054301,-0.670095,0.103371,0.313132,...,0.021602,0.014405,0.017049,13.993523,0.113546,-0.730467,13.820430,1.004931e+06,2000000.0,0.688228
1,234811,Rui Silva,Real Betis,Sporting CP,25/26,2025-07-01,0.105951,-0.593290,-0.000142,0.238576,...,0.000000,0.000000,0.006967,14.909505,0.113546,-0.730467,14.765329,2.585241e+06,4700000.0,0.597744
2,249303,Dennis Eckert Ayensa,Union SG,Standard Liège,25/26,2025-07-01,-0.000884,-0.379875,-0.000142,0.363332,...,0.038872,0.011109,0.008766,14.073177,0.113546,-0.730467,14.221522,1.500820e+06,1000000.0,-0.406012
3,263236,Kevin Danso,Lens,Tottenham,25/26,2025-07-01,0.092596,-0.256267,-0.000142,0.325060,...,0.000000,0.000000,0.021041,16.272818,0.113546,-0.730467,16.616453,1.646015e+07,25000000.0,0.417934
4,278343,Felix Uduokhai,FC Augsburg,Besiktas,25/26,2025-07-01,0.132659,-0.331272,0.103371,0.313132,...,0.000000,0.000000,0.007709,15.184325,0.113546,-0.730467,15.514826,5.470202e+06,5000000.0,-0.089878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3801,61336,Brian Hämäläinen,Zulte Waregem,KRC Genk,12/13,2012-08-31,-0.094364,0.001151,0.664464,0.258458,...,0.000000,0.000000,0.000000,13.197852,-0.158116,-0.730467,13.763338,9.491652e+05,1000000.0,0.052172
3802,104597,Bram Nuytinck,NEC Nijmegen,RSC Anderlecht,12/13,2012-08-31,0.105951,0.069155,0.716220,0.195334,...,0.088275,0.000000,0.000000,14.073177,-0.158116,-0.730467,15.015664,3.320626e+06,1750000.0,-0.640537
3803,132207,Matty Kennedy,Kilmarnock FC,Everton,12/13,2012-08-31,-0.107718,0.397578,0.711966,0.186388,...,0.000000,0.000000,0.000000,11.385553,-0.158116,-0.730467,12.478053,2.625123e+05,250000.0,-0.048837
3804,143559,Matija Nastasić,Fiorentina,Man City,12/13,2012-08-31,0.052533,0.280970,0.716220,0.198814,...,0.000000,0.000000,0.000000,14.826384,-0.158116,-0.730467,15.993583,8.829275e+06,15200000.0,0.543222


2533

In [42]:
import pandas as pd

# -----------------------------
# 1. File paths (edit if needed)
# -----------------------------
shap_df = pd.read_csv("player_shap_transfer_fee_55.csv")
games_df = pd.read_csv("player_game_scores_and_values.csv")
mlr_df  = pd.read_csv("mlr_local_explanations_per_player_55.csv")
JSONL_PATH = "players_llm.jsonl"  # <-- change to your actual jsonl filename

# -----------------------------
# 2. Load data
# -----------------------------


# JSONL: one JSON object per line
json_df = pd.read_json(JSONL_PATH, lines=True)

# Make sure player_id types match (very important!)
shap_df["player_id"] = shap_df["player_id"].astype(int)
games_df["player_id"] = games_df["player_id"].astype(int)
mlr_df["player_id"]   = mlr_df["player_id"].astype(int)
json_df["player_id"]  = json_df["player_id"].astype(int)

# -----------------------------
# 3. Unique player sets
# -----------------------------
shap_ids  = set(shap_df["player_id"].unique())
games_ids = set(games_df["player_id"].unique())
mlr_ids   = set(mlr_df["player_id"].unique())
json_ids  = set(json_df["player_id"].unique())

# players present in ALL FOUR datasets
common_ids = shap_ids & games_ids & mlr_ids & json_ids

# -----------------------------
# 4. Filter each file to intersection
# -----------------------------
shap_filtered  = shap_df[shap_df["player_id"].isin(common_ids)].copy()
games_filtered = games_df[games_df["player_id"].isin(common_ids)].copy()
mlr_filtered   = mlr_df[mlr_df["player_id"].isin(common_ids)].copy()
json_filtered  = json_df[json_df["player_id"].isin(common_ids)].copy()

# -----------------------------
# 5. Count keep/delete for each file
# -----------------------------
def report_counts(name, original_ids, filtered_df):
    total_players  = len(original_ids)
    keep_players   = len(set(filtered_df["player_id"].unique()))
    delete_players = total_players - keep_players

    print(f"=== {name} ===")
    print("Total players:    ", total_players)
    print("Kept (in all 4):  ", keep_players)
    print("Deleted (unique): ", delete_players)
    print()

report_counts("player_shap_transfer_fee_55.csv", shap_ids, shap_filtered)
report_counts("player_game_scores_and_values.csv", games_ids, games_filtered)
report_counts("mlr_local_explanations_per_player_55.csv", mlr_ids, mlr_filtered)
report_counts(JSONL_PATH, json_ids, json_filtered)

# -----------------------------e
# 6. Save filtered versions
# -----------------------------
shap_filtered.to_pickle("player_shap_transfer_fee_321.pkl")
games_filtered.to_pickle("player_game_scores_and_values_321.pkl")
mlr_filtered.to_pickle("mlr_local_explanations_per_transfer_321.pkl")

# JSONL: preserve line-by-line JSON format
json_filtered.to_json(
    "players_intersection_321.jsonl",
    orient="records",
    lines=True
)

print("Filtered files saved with only players present in ALL FOUR datasets.")


=== player_shap_transfer_fee_55.csv ===
Total players:     2537
Kept (in all 4):   1451
Deleted (unique):  1086

=== player_game_scores_and_values.csv ===
Total players:     24287
Kept (in all 4):   1451
Deleted (unique):  22836

=== mlr_local_explanations_per_player_55.csv ===
Total players:     2533
Kept (in all 4):   1451
Deleted (unique):  1082

=== players_llm.jsonl ===
Total players:     32601
Kept (in all 4):   1451
Deleted (unique):  31150

Filtered files saved with only players present in ALL FOUR datasets.


In [43]:
readshap = pd.read_pickle("player_shap_transfer_fee_321.pkl")
readshap

,player_id,transfer_year,transfer_season,from_club_name,to_club_name,transfer_date,transfer_fee,log_transfer_fee,pred_log_transfer_fee,pred_transfer_fee,...,shap_sub_position_Left-Back,shap_sub_position_Right Midfield,shap_sub_position_Right Winger,shap_sub_position_Right-Back,shap_sub_position_Second Striker,shap_foot_both,shap_foot_left,shap_foot_right,shap_log_market_value_in_eur,shap_transfer_year
2,7161,2016,15/16,Bor. Dortmund,Bor. M'gladbach,2016-01-01,8000000.0,15.894952,15.233380,4.128317e+06,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.162248,0.032758
3,7825,2015,15/16,Bayern Munich,Napoli,2015-07-01,2000000.0,14.508658,14.747121,2.538594e+06,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.109791,-0.062724
5,11530,2018,18/19,Galatasaray,R. Strasbourg,2018-08-17,750000.0,13.527830,14.352508,1.710862e+06,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.083098,0.034199
6,12029,2016,15/16,Angers SCO,Club Brugge,2016-01-01,1000000.0,13.815512,13.439337,6.864830e+05,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,-0.089390,0.033095
7,12282,2018,18/19,Man Utd,Ajax,2018-07-17,16000000.0,16.588099,16.649675,1.701618e+07,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.417571,0.034263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2517,657387,2018,18/19,Linense,Braga U19,2018-07-01,100000.0,11.512935,12.628443,3.051146e+05,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,-0.332188,0.097155
2519,671915,2019,19/20,Newell's II,FC Sion,2019-08-22,3100000.0,14.946913,13.023510,4.529378e+05,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.01282,-0.321582,0.256850
2521,673492,2019,19/20,São Paulo,Benfica B,2019-09-02,7600000.0,15.843659,13.380204,6.470664e+05,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.01349,-0.324140,0.297623
2530,748601,2019,18/19,Ros. Central II,Boca Juniors II,2019-01-31,1800000.0,14.403298,13.644279,8.426268e+05,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,-0.289927,0.304101


In [44]:
readgamescore = pd.read_pickle("player_game_scores_and_values_321.pkl")
readgamescore

,player_id,player_name,time,universal_score_100,market_value
39574,7161,Jonas Hofmann,2012-12-16,19.047619,200000.0
39575,7161,Jonas Hofmann,2013-04-06,23.809524,200000.0
39576,7161,Jonas Hofmann,2013-04-27,19.047619,200000.0
39577,7161,Jonas Hofmann,2013-08-03,47.619048,500000.0
39578,7161,Jonas Hofmann,2013-08-10,33.333333,500000.0
...,...,...,...,...,...
1677577,861410,Arda Güler,2025-03-01,19.047619,45000000.0
1677578,861410,Arda Güler,2025-03-15,19.047619,45000000.0
1677579,861410,Arda Güler,2025-03-29,25.960061,45000000.0
1677580,861410,Arda Güler,2025-04-01,33.333333,45000000.0


In [45]:
readmlr = pd.read_pickle("mlr_local_explanations_per_transfer_321.pkl")
readmlr

,player_id,player_name,from_club_name,to_club_name,transfer_season,transfer_date,coef_height_in_cm_c,coef_age_at_transfer_c,coef_contract_years_left,coef_from_foreigners_percentage,...,coef_goals_per90,coef_assists_per90,coef_cards_per90,coef_log_market_value_in_eur,coef_transfer_year_c,coef_intercept,pred_log_transfer_fee,pred_transfer_fee,actual_transfer_fee,residual_log
0,7161,Jonas Hofmann,Bor. M'gladbach,B. Leverkusen,23/24,2023-07-05,-0.094364,-0.562487,0.206457,0.267404,...,0.059289,0.033888,0.006685,15.648127,0.071752,-0.730467,15.823265,7.446576e+06,10000000.0,0.294831
1,7825,Pepe Reina,Bayern Munich,Napoli,15/16,2015-07-01,0.065888,-0.698297,0.517845,0.286789,...,0.000000,0.000000,0.048733,14.247347,-0.095425,-0.730467,14.414080,1.819514e+06,2000000.0,0.094578
2,11530,Lionel Carole,Galatasaray,R. Strasbourg,18/19,2018-08-17,-0.040947,-0.297670,0.355770,0.264919,...,0.000000,0.009811,0.000000,14.247347,-0.032733,-0.730467,14.488484,1.960057e+06,750000.0,-0.960656
3,12029,Ludovic Butelle,Angers SCO,Club Brugge,15/16,2016-01-01,0.065888,-0.692096,0.491754,0.283807,...,0.000000,0.000000,0.000000,13.197852,-0.074528,-0.730467,13.284888,5.882389e+05,1000000.0,0.530622
4,12282,Daley Blind,Man Utd,Ajax,18/19,2018-07-17,-0.040947,-0.371274,0.411922,0.343948,...,0.012025,0.008019,0.009491,15.959001,-0.032733,-0.730467,16.205038,1.090834e+07,16000000.0,0.383061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,657387,Fabiano,SC Braga,Moreirense,23/24,2023-08-09,-0.107718,-0.009450,0.149738,0.286789,...,0.000000,0.000000,0.046899,14.247347,0.071752,-0.730467,14.783880,2.633646e+06,700000.0,-1.325044
2343,671915,Enzo Barrenechea,Juventus,Aston Villa,24/25,2024-07-01,0.039179,0.011951,0.051614,0.368302,...,0.004000,0.002668,0.031573,15.122672,0.092649,-0.730467,15.905692,8.086380e+06,8000000.0,-0.010740
2345,673492,Morato,Benfica,Nottm Forest,24/25,2024-08-30,0.119305,0.007751,0.250273,0.313132,...,0.000000,0.003431,0.020305,15.784830,0.092649,-0.730467,16.805318,1.988186e+07,11000000.0,-0.591913
2425,748601,Gastón Ávila,Royal Antwerp,Ajax,23/24,2023-08-22,-0.014238,0.100958,0.251549,0.248517,...,0.000000,0.012922,0.007647,14.394606,0.071752,-0.730467,14.986349,3.224697e+06,12500000.0,1.354890
